InterConnect: Identify clients at risk of chrun

___________________________________________________________________________________________________________
EDA

InterConnect has supplied us with 4 csv files about their customers, and the services they use from InterConnect. We have been asked to forecast churn for customers of InterConnect, specifically our goal is to pinpoint which clients are most likely to leave, allowing InterConnect to intervene with targeted promotions for retaining business.

Data is loaded, then cleaned while we explore features. Missing values will be filled in, some naming conventions are standardized, data type corrections, incorrect data values corrected, some feature engineering is conducted, conversion of yes/ no to binary 1/0, data files were merged, user behavior was explored, feature distributions were compared, and comparing churned clients to existing one was executed.

In [ ]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# load the csv files
# what are the csv files?

# directory location
folder_path = '/datasets/final_provider/'

# store the .csv file names
csv_files = [every_file for every_file in os.listdir(folder_path) if every_file.endswith('.csv')]

# snapshot of the files
print(csv_files)

In [ ]:
# load the csv files
df_0 = pd.read_csv(folder_path + csv_files[0])
df_1 = pd.read_csv(folder_path + csv_files[1])
df_2 = pd.read_csv(folder_path + csv_files[2])
df_3 = pd.read_csv(folder_path + csv_files[3])

In [ ]:
# df sizes
print(df_0.shape, df_1.shape, df_2.shape, df_3.shape)

**Each Dataframe is connected through the 'customerID', which is a unique customer identifier found in each dataframe** <br> df's 0 and 2 have every user's (customerID), we will merge into one of them


In [ ]:
# merge all dataframes together
df = df_0.merge(df_1, 
                how='left', 
                on='customerID')

df = df.merge(df_2,
              how='left', 
              on='customerID')

df = df.merge(df_3, 
              how='left', 
              on='customerID')
df.info()

thoughts
___
* 7043 rows or different customer contracts, 20 columns
* some missing values
* columns include: unique ID, start and end date of contract, some billing information, how much they pay, and user behavior stats
* only two numeric columns? data types investigate
* Not all users, use all features. some have only one line or dont use internet etc.

In [ ]:
# visualize it 
df.head()

thoughts
____
Lots of columns to make binary, and some potential feature engineering as I learn what columns means.

more specific thoughts
+ dates will need to be explored, feature engineered? maybe create some sort of loyalty ranking/system
+ type, if two types, then make binary
+ multiple lines: no=0, missing=0, yes=1
+ paperlessbilling turn into binary
+ payment method, explore how many different options there are
+ monthly charges turn into float
+ totalcharges, not sure what yet
+ gender, make binary
+ senior citizen might be fine
+ partner means what? make binary
+ dependents make binary
+ internet type, how mnay options
+ onlinesecurity, onlinebackup, online protection, tech support, deviceProtection, streaming movies, streaming tv make binary?

In [ ]:
# gender column name doesn't follow the naming convention
df = df.rename(columns={'gender':'Gender'})

In [ ]:
# convert dates to date format
# begin date looks complete, no missing values, if no error occurs then they are all the right format
df['BeginDate'] = pd.to_datetime(df['BeginDate'])

In [ ]:
# total charges is current object type, contains non-numeric values

# convert non-numeric entries to missing values
test = pd.to_numeric(df['TotalCharges'], errors='coerce')

# locate where the incorrect entries are
nan_mask = test.isna()
idx_nan = df[nan_mask].index.to_list()

# who are they?
df.loc[nan_mask]

thoughts
___ 
- The incorrect total charge entries appear to be blank. 
- They also all were signed up on the most recent day. they are brand new customers
- we should make total charges 0 because they perhaps havent paid a bill yet.

In [ ]:
# change the NaN's to 0 and convert the totalcharge column to numeric

# change nan values to 0
df.loc[nan_mask, 'TotalCharges'] = 0

#  change the column dtype
df['TotalCharges'] = df['TotalCharges'].astype('float64')

In [ ]:
# end date column has 'no' values for clients that have no churned we need to handle these values
df['EndDate'].value_counts()

thoughts
____
+ out of our ~7000 rows 5100 haven't churned!
+ and every client who has churned, churned on only 4 different days? So according to this data our customers only cancel their service from October to January? Looks like we should market promotions starting around Labor Day - New Years. Something about summer ending (I assume northern hemisphere) makes people more aware of their billing subscriptions or the realative quality of them? Is this when our competition offers promotions?

In [ ]:
# replace 'No' with missing values
df['EndDate'] = df['EndDate'].replace('No', np.nan)

# convert the column to datetime format
df['EndDate'] = pd.to_datetime(df['EndDate'])

In [ ]:
# create the target feature column of churned 
df['Churn'] = df['EndDate'].notnull().astype(int)

**Deeper exploration:**
___

- CustomerID should be unique, but is it?
- What is the range of Sign up dates?
- What percent of clients have churned?
- Popular join months?

In [ ]:
# is customerID unique?
print((len(df)) == (df.customerID.nunique()))

In [ ]:
# what is the range of sign up date?
print(f"Earliest sign up date is: {df['BeginDate'].min()}")
print(f"Most recent sign up date is: {df['BeginDate'].max()}")
print()
print()
sns.set_style('whitegrid')
df['BeginDate'].hist(bins=80)
plt.title('Distribution of signup dates for InterConnect Clients')
plt.xlabel('Signup date')
plt.ylabel('Count')
plt.show()


thoughts
___
+ whatever happened in the middle of 2018, do that!
+ Data begins in October 2013, and ends on February 2020.
+ The range of EndDate's was easy to view in value_counts. Before October 2019 InterConnect hasn't lost any clients, starting October 2019 InterConnect began losing between 458-485 each month. No wonder they are so interested in who is about to churn. It is a new business problem for them. 

In [ ]:
# how many signups occurred during the churn?
start_date_for_count = pd.to_datetime('2019-10-01')

# bool mask to filter dates
signups_since_oct1_mask = df['BeginDate'] >= start_date_for_count

#signup count
print(signups_since_oct1_mask.sum())

In [ ]:
thoughts
____
Oh no! that's well less than churn. thats why they hired us.

In [ ]:
# what is the monthly revenue loss from churned clients
# The difference in churn - signups, multiplied by the average of those who churned. 
month_loss = (df['Churn'].sum() - signups_since_oct1_mask.sum()) * (df[df['Churn']==1]['MonthlyCharges'].mean())
print(f"This isn't exact as those who joined, have different bills from those who left, but InterConnect is approximately losing ${month_loss:.2f} a month in gross revenue")

In [ ]:
# what percent of clients have churned
print(f"{(df['Churn'].sum() / len(df))*100:.4f} % of subscriptions have been canceled")

In [ ]:
# popular join months, create a column recording the month
df['JoinMonth'] = df['BeginDate'].dt.month
print(df['JoinMonth'].value_counts())

sns.countplot(x='JoinMonth', data=df)
plt.title('Member signups per month')
plt.xlabel('Month')
plt.ylabel('Count')
plt.show()


thoughts
____
+ about a quarter of all subscriptions have canceled
+ February is the most popular month for signups with 804, May is the least popular at 494

In [ ]:
# is this every february or a specific february?
# create a column for join year
df['JoinYear'] = df['BeginDate'].dt.year

sns.set_style('whitegrid')
february_signups = df[df['JoinMonth'] == 2].copy()
february_signups['JoinYear'].value_counts().plot.bar()
plt.title('February Signups by year')
plt.xlabel('Year')
plt.ylabel('Count')
plt.show()

thoughts
___
February is  not special, february 2014 was. Did a Valentines Day, presidents day or Superbowl promotion occur?

In [ ]:
# how long has each subscription existed?

# most recent possible date
pull_date = pd.to_datetime('2020-02-01')

# whether a customer has churned, na is False or not churned
condition = df['EndDate'].notna()

# if churned, then whats the length
value_if_true = (df['EndDate'] - df['BeginDate']).dt.days

# if active, then pull_date - BeginDate
value_if_false = (pull_date - df['BeginDate']).dt.days

# populate the 'MemberLengthDays' column with the length of contract
df['MemberLengthDays'] = np.where(condition, value_if_true, value_if_false)

# make sure the value is 'int'
df['MemberLengthDays'] = df['MemberLengthDays'].astype(int)



In [ ]:
print(f"Newest member contract is: {df['MemberLengthDays'].min()} days old")
print(f"Longest member contract is: {df['MemberLengthDays'].max()} days old")

In [ ]:
with pd.option_context('display.max_rows', 200):
    print(df[df['Churn']==1]['MemberLengthDays'].value_counts())

thoughts
____
Churn occurs in month intervals. We have month to month contracts and annual contracts. Most churn happened after one month (30 or 31 days). <br>The top 5 most common churn intervals are:<br>1. 1 month<br>2. 1 month<br>3. 2 months<br>4. 5 months<br>5. 3 months

In [ ]:
# manual count for how many churns were in the top5 = 630
print(f"{630/(df['Churn'].sum()) *100}% of churn occurred within the first 5 months")

In [ ]:
# what percentage of churn comes from clients who have lasted longer than a year?
len(df[(df['Churn']==1) & (df['MemberLengthDays']>365)])/ df['Churn'].sum() *100

thoughts
___
InterConnect offers month to month and 1 year / two years plans. Honestly viewing churn rates after 5 months (top five churn time lines) and 1 year, it didn't spark the insight I had hoped. ~55% (the opposite of 44.5) of churn occurs within the first year. There could be some potential to offer promotions here althought customers who opt for month to month phone plans might be too transitional to really lock into telecommunication plans. This would require more customer data.

In [ ]:
# monthly charges for churned or not

sns.violinplot(x='Churn', y='MonthlyCharges', data=df, inner='quartile')
plt.title('Monthly bills for active and former customers')
plt.xlabel('Churn status')
plt.xticks(ticks=[0, 1], labels=['Not Churned (0)', 'Churned (1)'])
plt.ylabel('Monthly Bill ($)')
plt.show()

In [ ]:
print(f"Average Bill for Churned clients: {df[df['Churn']==1]['MonthlyCharges'].mean():.2f}")
print(f"Average Bill for Existing clients: {df[df['Churn']==0]['MonthlyCharges'].mean():.2f}")

In [ ]:
# where would the average non-churned bill( 61.27) land on the churned plot? it is above 25% but not by a lot

print(f"25% {df[df['Churn']==1]['MonthlyCharges'].quantile(.25)}")
print(f"26% {df[df['Churn']==1]['MonthlyCharges'].quantile(.26)}")
print(f"27% {df[df['Churn']==1]['MonthlyCharges'].quantile(.27)}")
print(f"28% {df[df['Churn']==1]['MonthlyCharges'].quantile(.28)}")
print(f"29% {df[df['Churn']==1]['MonthlyCharges'].quantile(.29)}")

thoughts
___
+ Average Bill for Churned customers is higher than not churned.
+ The average bill for existing clients (the larger cohort) would only be the 27th percentile for churned clients. We will continue with a machine learning problem because this is a school assignment however, it is clear that the higher your monthly bill is the more likely they will leave as customers.

In [ ]:
# who cancels their subscriptions?
# groupby chart of percentages
# do this after we convert stuff to binary

In [ ]:
# convert binary column values
def encode_column(columns_to_encode, dataframe):
    """
    Converts multiple specified columns into machine learning model ready values:
    if a column has less than 3 options then:
    'Yes' maps to 1 and 'No' or missing map to 0
    for columns with 3 or more options then one hot encoding will be done.

    Args: 
    columns_to_encode (str or list of strings): the columns we want to convert
    dataframe (pd.DataFrame): pass the dataframe to modify

    Returns:
    Dataframe with updated data

    Raises:
    TypeError: If 'dataframe' is not a pandas DataFrame or 'columns_to_encode' is not a string or list of strings.
    KeyError: If the column is not part of the dataframe
    """

    # make sure inputs are correct
    if not isinstance(dataframe, pd.DataFrame):
        raise TypeError(f"Expected a pd.DataFrame, but got {type(dataframe)}")

    # confirm columns_to_encode is a list for iteration
    if isinstance(columns_to_encode, str):
        columns_list = [columns_to_encode]
        
    elif isinstance(columns_to_encode, (list, pd.Series, pd.Index)):
        columns_list = list(columns_to_encode)
        
    else:
        raise TypeError(f"Expected 'columns_to_encode' to be a string or a list/Series/Index of strings, but got {type(columns_to_encode)}")

    # map entries 
    mapping = {'Yes': 1, 
               'No': 0, 
               np.nan: 0, 
               'Male': 0,
               'Female': 1,
               'Fiber optic': 0, 
               'DSL': 1}

    # loop through columns to encode
    for col_name in columns_list:
        if col_name not in dataframe.columns:
            raise KeyError(f"Column '{col_name}' not found in the DataFrame.")
            
        # print status
        print(f"\n\nColumn: '{col_name}'")
        print(f"Original data type: {dataframe[col_name].dtype}")
        print(f"Original value counts (w/ NaN):\n{dataframe[col_name].value_counts(dropna=False)}\n")

        # determine which type of encoding needs to be executed
        unique_count = dataframe[col_name].nunique()

        # binary encoding
        if unique_count <= 2:
            dataframe[col_name] = dataframe[col_name].replace(mapping)

            # output status
            print(f"Column data type is now: {dataframe[col_name].dtype}")
            print(f"No or missing is now 0, Yes is 1\n{dataframe[col_name].value_counts(dropna=False)}")

        # one hot encoding
        else:
            # use pd.get_dummies to One Hot Encode
            dataframe = pd.get_dummies(dataframe, columns=[col_name], prefix=col_name, dtype=int, dummy_na=False)

            # print dummy columns
            print(f"New columns created for '{col_name}':")
    
            dummy_cols = [col for col in dataframe.columns if col.startswith(f'{col_name}_')]
                
            if dummy_cols:
                print(dataframe[dummy_cols].head())
                print(f"Data types of new dummy columns: {dataframe[dummy_cols].dtypes.unique()}")

    return dataframe

In [ ]:
part_one = list(range(3,6))
part_two = list(range(8,20))
columns_list = part_one + part_two
print(columns_list)

In [ ]:
df = encode_column(df.columns[columns_list], df)

In [ ]:
df.groupby('Churn').mean().T

thoughts
___

Feature use where churn is more likely:
- paperless billing, although popular for both
- multiple lines, slightly
- senior citizen, this could be age related, death, moving into assisted facility or into family home
- streamingtv, and streaming movies appear to be the exact same
- month to month bill (extreme correlation)
- payment by electronic check, not sure what this is but relates to churn

**88.6% of churn comes from month to month clients**

Features where churn is less likely:
- has partner
- has dependents
- has internet
- has internet backup
- has internet security
- device protection
- tech support
- one year or two year contract
- any payment method that isn't electronic check

**The average lifetime charges for churned clients is 1531 USD at an average of 74/month**

**The average lifetime charges for existing clients is 2550 USD at an averge of 61/month**

Smaller bills lead to greater lifetime earning. We do not have information about operation costs for InterConnect. InterConnect should investigate what the operational cost is to host an additional line for phone or internet service. So far it appears that people do not cancel less expensive bills, and their clients dont seem to cancel annual contracts. They should definitely promote the annual contracts. If someone's lifestyle is too transitional for an annual contract then perhaps there isn't as much you can do to retain their business (this is speculation). The features which seem to contribute toward business retention also appear to be more stable lifestyle.


In [ ]:
# most of the churn is month to month clients, and most of the churn has been recent. When did month-to-month begin?
month_to_month_clients = df[df['Type_Month-to-month'] == 1]

# first month to month contract date
print(month_to_month_clients['BeginDate'].min())

thoughts
____
Month to month contracts are not new, why the sudden churn?

In [ ]:
# visualizing churn correlation in descending order
# empty dictionary
corr_dict ={}
# loop through columns, skipping a few
for col in df.columns:
    if col in ['Churn', 'customerID', 'EndDate', 'BeginDate', 'Type']:
        continue
    # store the correlation info in the dictionary
    corr_dict[col] = df['Churn'].corr(df[col])

corr_series = pd.Series(corr_dict)
print(corr_series.sort_values(ascending=False))

thoughts
___
We saw a lot of this information earlier when we presented the differences between churn and unchurned clients. 

In [ ]:
plt.figure(figsize=(14, 14))
# Select only numerical columns for correlation
numerical_df = df.select_dtypes(include=np.number)
sns.heatmap(numerical_df.corr(), annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Matrix of Numerical Features')
plt.show()

thoughts
____
similiar info as before, compounds the notion that gender is no correlated to any feature, not just churned

In [ ]:

# look for outliers
df[df['Churn']==0]['MonthlyCharges'].hist(bins=100)


In [ ]:
# look for outliers
df[df['Churn']==1]['MonthlyCharges'].hist(bins=100)


In [ ]:
# remove less meaningful columns
col_to_drop = ['BeginDate', 'EndDate', 'TotalCharges', 'Gender', 'JoinMonth', 'JoinYear']
df.drop(columns=col_to_drop, inplace=True)

In [ ]:
df.groupby(['Type_Month-to-month', 'Churn']).mean().T

thoughts
___
+ month to month (m2m) is more likely to be paperless
+ if a senior citizen is m2m then they are more likely to churn
+ partner or dependents less likely to churn
+ having internet service discourages churn
+ any device upgrade/ extra services (device protection etc.) less likely to churn, maybe promo?
+ and electronic checks still mean churn, is this payment method annoying or something?

In [ ]:
df.groupby(['Type_One year', 'Churn']).mean().T

thoughts
____
+ higher monthly bill means more churn
+ partner yes, churn yes, one-year no, is a rare combo
+ dependents yes, churn yes, one-year no is a rare combo
+ if you have device upgrades you are more likely to churn

In [ ]:
df.groupby(['Type_Two year', 'Churn']).mean().T

thoughts
___
+ two year contracts were not as dramatic.
+ multiple lines more likely to churn
+ partner + two year contract = not churned
+ mailed check means not churned essentially
+ device upgrades more likely to be a two year contract

In [ ]:
# final column name adjustments
df = df.rename(columns={'customerID': 'CustomerID', 
                   'Type_Month-to-month':'Type_Month_To_Month', 
                   'Type_One year':'Type_One_Year', 
                   'Type_Two year':'Type_Two_Year', 
                   'PaymentMethod_Bank transfer (automatic)':'PaymentMethod_Bank_Transfer_Automatic', 
                   'PaymentMethod_Credit card (automatic)': 'PaymentMethod_Credit_Card_Automatic',
                   'PaymentMethod_Electronic check': 'PaymentMethod_Electronic_Check',
                   'PaymentMethod_Mailed check': 'PaymentMethod_Mailed_Check'})

conclusion
____
Data has been cleaned and explored. CustomerID is unique for each customer, and there are no duplicates. InterConnect is concerned about churn, and it is understandable why. All of the churn has occurred in the last four months, with around 460 per month, totaling 1869 customers. During that time only 938 signups occurred. The difference of ~700 clients is a revenue decrease of approximately $69,000 a month. InterConnect saw an dramatic increase of signups in 2018. They should explore what business decisions or events around this time, could be recycled for today. Signups are lowest during summer months. Most of the clients who churn are 'month-to-month' users (88.6% of churn is from Month to month). The month to month contract has been around for years. It wasn't simply they offered a new short term plan, and people churn now. 

Clients who churn have a higher average monthly bill. I did not conduct a statistic test to confirm an event, however the 27% percentile of churned clients pays a higher bill than non-churned clients. There are several features that appear to correlate to churn. These include: senior citizen, higher monthly bills, electronic checks, and month-to-month subscriptions.

Features where churn was less likely: has partner, has dependents, has interent service, has internet backup, security, device protection, or tech support. Being on an annual style contract as discourages churn.


The average lifetime charges for churned clients is 1531 USD at an average of 74/month

The average lifetime charges for existing clients is 2550 USD at an averge of 61/month

Customers with smaller monthly bills tend to have longer tenures, leading to greater overall lifetime revenue. We do not have information about operation costs for InterConnect. InterConnect should investigate what the operational cost is to host an additional line for phone or internet service. So far it appears that people do not cancel less expensive bills, and their clients dont seem to cancel annual contracts. 

They should definitely promote the annual contracts. If someone's lifestyle is too transitional for an annual contract then perhaps there isn't as much you can do to retain their business (this is speculation). The features which seem to contribute toward business retention also appear to be more stable lifestyle.

questions:
+ Can we identify distinct segments or 'profiles' within the 'Month-to-month' customer base (with clustering) who exhibit retention patterns similar to annual contract holders, potentially for targeted promotional offers to convert them?
+ There are also feature that promote non-churn, would bundling or promoting these features within less expensive plans (or offering them at reduced rates) significantly increase customer longevity, especially given the observed retention of customers with smaller bills?
+ Is there a monthly bill threshold where going beyond that number promotes churn? 
+ All the churn has occurred recently, do we have new competition that wasn't previously available? Has the quality of service offered been affected in the last four months? whether due to uncontrolable stuff like weather or service outages?

plan of action:

We identified several features which may indicate churn status. Our modeling will pair these together creating member 'profiles' and identify those who are expected to churn, based on similiarities to previous churned clientele.

- split the data for features/ target, and for training validation sets to ensure robust model development and unbiased performance evaluation.
- based on EDA insights and initial correlation analysis, select the most relevant features to be modeled on, to speed up training, and improve model interpretability/generalization
- apply appropriate scaling for continuous numerical features, includes member length, monthly charges (fit/ transform training, and just transform testing)
- choose models suited for both our data and the type of question we want to answer. Beginning with more interpretable models, to predict customer churn
- train selected models and evaulate performances using the appropriate metrics, then perform hyperparameter tuning to optimize the best performing models
